In [8]:
## WITHOUT TKINTER

In [9]:
import cv2
import numpy as np
from quickdraw import QuickDrawData
from skimage.metrics import structural_similarity as ssim
import time

# --- 1. QuickDraw Helper Functions ---
def get_quickdraw_reference(category):
    qd = QuickDrawData()
    if category not in qd.drawing_names:
        raise ValueError("Invalid category.")
    return qd.get_drawing(category)

def render_drawing_to_image(drawing, size=(28, 28)):
    img = np.ones((255, 255), dtype=np.uint8) * 255
    for stroke in drawing.strokes:
        for i in range(len(stroke) - 1):
            x1, y1 = stroke[i]
            x2, y2 = stroke[i + 1]
            cv2.line(img, (x1, y1), (x2, y2), color=0, thickness=2)
    img_resized = cv2.resize(img, size, interpolation=cv2.INTER_AREA)
    return img_resized

def preprocess_canvas(canvas):
    gray = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)
    resized = cv2.resize(thresh, (28, 28))
    return resized

def compare_drawings(user_img, ref_img):
    user = user_img / 255.0
    ref = ref_img / 255.0
    score, _ = ssim(user, ref, full=True, data_range=1.0)
    return score

# --- 2. Main Function ---
def main():
    # Get drawing category from user
    category = input("Enter a category to draw (e.g., cat, tree, airplane): ").strip().lower()

    try:
        drawing = get_quickdraw_reference(category)
        reference_img = render_drawing_to_image(drawing)
    except ValueError as e:
        print(e)
        return

    print("Draw in the air using a yellow marker. Press 's' to submit. Press 'c' to clear. Press 'q' to quit.")

    # Open webcam
    cap = cv2.VideoCapture(0)
    time.sleep(2)  # Allow camera to warm up

    canvas = np.ones((480, 640, 3), dtype=np.uint8) * 255

    # Blue color HSV range
    lower_color = np.array([20, 100, 100])
    upper_color = np.array([40, 255, 255])
    kernel = np.ones((5, 5), np.uint8)

    prev_center = None

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        #mask = cv2.inRange(hsv, lower_color, upper_color)
        mask = cv2.inRange(hsv, lower_color, upper_color)
        mask = cv2.erode(mask, kernel, iterations=2)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        mask = cv2.dilate(mask, kernel, iterations=1)
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if contours:
            largest = max(contours, key=cv2.contourArea)
            if cv2.contourArea(largest) > 100:
                x, y, w, h = cv2.boundingRect(largest)
                center = (x + w // 2, y + h // 2)

                cv2.circle(frame, center, 5, (0, 255, 0), -1)

                if prev_center:
                    cv2.line(canvas, prev_center, center, (0, 0, 0), 5)
                prev_center = center
        else:
            prev_center = None

        cv2.imshow("Air Drawing - Webcam Feed", frame)
        cv2.imshow("Air Canvas - Press 's' to Submit", canvas)

        key = cv2.waitKey(1) & 0xFF

        if key == ord('s'):
            break
        elif key == ord('c'):
            canvas[:] = 255
        elif key == ord('q'):
            print("Exiting.")
            cap.release()
            cv2.destroyAllWindows()
            return

    cap.release()
    cv2.destroyAllWindows()

    # Process user drawing
    user_img = preprocess_canvas(canvas)

    # Show user drawing
    cv2.imshow("Your Drawing (Processed)", user_img)  
    #--  commenting as google does not allow this fubction
    #cv2_imshow(user_img)
    cv2.waitKey(1000)
    cv2.destroyAllWindows()

    # Compare
    score = compare_drawings(user_img, reference_img)
    print(f"Similarity to '{category}' reference: {score:.2f}")
    if score > 0.5:
        print("✅ Drawing matches the category!")
    else:
        print("❌ Drawing does not match. Try again!")

# --- Run the program ---
if __name__ == "__main__":
    main()

    

Enter a category to draw (e.g., cat, tree, airplane): airplane
downloading airplane from https://storage.googleapis.com/quickdraw_dataset/full/binary/airplane.bin
download complete
loading airplane drawings
load complete
Draw in the air using a yellow marker. Press 's' to submit. Press 'c' to clear. Press 'q' to quit.
Exiting.


In [ ]:
## WITH TKINTER LIBRARY

In [12]:
import cv2
import numpy as np
import threading
import tkinter as tk
from tkinter import simpledialog, messagebox, Toplevel, Label
from PIL import Image, ImageTk
from quickdraw import QuickDrawData
from skimage.metrics import structural_similarity as ssim

# --- QuickDraw Helper Functions ---
def get_quickdraw_reference(category):
    qd = QuickDrawData()
    if category not in qd.drawing_names:
        raise ValueError("Invalid category.")
    return qd.get_drawing(category)

def render_drawing_to_image(drawing, size=(28, 28)):
    img = np.ones((255, 255), dtype=np.uint8) * 255
    for stroke in drawing.strokes:
        for i in range(len(stroke) - 1):
            x1, y1 = stroke[i]
            x2, y2 = stroke[i + 1]
            cv2.line(img, (x1, y1), (x2, y2), color=0, thickness=2)
    img_resized = cv2.resize(img, size, interpolation=cv2.INTER_AREA)
    return img_resized

def preprocess_canvas(canvas):
    gray = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)
    resized = cv2.resize(thresh, (28, 28))
    return resized

def compare_drawings(user_img, ref_img):
    user = user_img / 255.0
    ref = ref_img / 255.0
    score, _ = ssim(user, ref, full=True, data_range=1.0)
    return score

# --- App Class ---
class AirDrawingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Air Drawing Recognizer")

        self.category = simpledialog.askstring("Input", "Enter a category (e.g., cat, tree):", parent=self.root)
        if not self.category:
            self.root.destroy()
            return

        try:
            self.reference_drawing = get_quickdraw_reference(self.category.lower())
            self.reference_img = render_drawing_to_image(self.reference_drawing)
        except ValueError as e:
            messagebox.showerror("Error", str(e))
            self.root.destroy()
            return

        self.canvas_frame = tk.Label(self.root)
        self.canvas_frame.pack()

        self.btn_frame = tk.Frame(self.root)
        self.btn_frame.pack()

        self.submit_btn = tk.Button(self.btn_frame, text="Submit", command=self.submit)
        self.submit_btn.grid(row=0, column=0)

        self.clear_btn = tk.Button(self.btn_frame, text="Clear", command=self.clear_canvas)
        self.clear_btn.grid(row=0, column=1)

        self.quit_btn = tk.Button(self.btn_frame, text="Quit", command=self.quit_app)
        self.quit_btn.grid(row=0, column=2)

        self.cap = cv2.VideoCapture(0)
        self.drawing_canvas = np.ones((480, 640, 3), dtype=np.uint8) * 255

        self.lower_color = np.array([20, 100, 100])
        self.upper_color = np.array([40, 255, 255])
        self.kernel = np.ones((5, 5), np.uint8)
        self.prev_center = None

        self.running = True
        self.update_frame()

    def update_frame(self):
        if not self.running:
            return

        ret, frame = self.cap.read()
        if not ret:
            return

        frame = cv2.flip(frame, 1)
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        mask = cv2.inRange(hsv, self.lower_color, self.upper_color)
        mask = cv2.erode(mask, self.kernel, iterations=2)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, self.kernel)
        mask = cv2.dilate(mask, self.kernel, iterations=1)

        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            largest = max(contours, key=cv2.contourArea)
            if cv2.contourArea(largest) > 100:
                x, y, w, h = cv2.boundingRect(largest)
                center = (x + w // 2, y + h // 2)
                cv2.circle(frame, center, 5, (0, 255, 0), -1)
                if self.prev_center:
                    cv2.line(self.drawing_canvas, self.prev_center, center, (0, 0, 0), 5)
                self.prev_center = center
        else:
            self.prev_center = None

        display = np.hstack((frame, self.drawing_canvas))
        img = Image.fromarray(cv2.cvtColor(display, cv2.COLOR_BGR2RGB))
        imgtk = ImageTk.PhotoImage(image=img)

        self.canvas_frame.imgtk = imgtk
        self.canvas_frame.configure(image=imgtk)
        self.root.after(10, self.update_frame)

    def submit(self):
        user_img = preprocess_canvas(self.drawing_canvas)
        score = compare_drawings(user_img, self.reference_img)
        result = f"Similarity to '{self.category}': {score:.2f}\n"
        result += "✅ Match!" if score > 0.5 else "❌ Try again!"
        self.show_comparison_window(user_img, self.reference_img, result)

    def clear_canvas(self):
        self.drawing_canvas[:] = 255

    def quit_app(self):
        self.running = False
        self.cap.release()
        self.root.destroy()

    def show_comparison_window(self, user_img, ref_img, result_text):
        comp_win = Toplevel(self.root)
        comp_win.title("Comparison")

        def img_to_tk(img_array):
            img = Image.fromarray(img_array)
            img = img.resize((100, 100), Image.NEAREST)
            return ImageTk.PhotoImage(img)

        # Convert grayscale OpenCV image to RGB for Tkinter display
        user_display = cv2.cvtColor(user_img, cv2.COLOR_GRAY2RGB)
        ref_display = cv2.cvtColor(ref_img, cv2.COLOR_GRAY2RGB)

        tk_user_img = img_to_tk(user_display)
        tk_ref_img = img_to_tk(ref_display)

        tk.Label(comp_win, text="Your Drawing").grid(row=0, column=0)
        tk.Label(comp_win, text="Reference").grid(row=0, column=1)
        tk.Label(comp_win, image=tk_user_img).grid(row=1, column=0)
        tk.Label(comp_win, image=tk_ref_img).grid(row=1, column=1)
        tk.Label(comp_win, text=result_text, font=('Arial', 12)).grid(row=2, column=0, columnspan=2)

        # Keep a reference to the images so they aren't garbage collected
        comp_win.tk_user_img = tk_user_img
        comp_win.tk_ref_img = tk_ref_img

# --- Run ---
if __name__ == "__main__":
    root = tk.Tk()
    app = AirDrawingApp(root)
    root.mainloop()


loading cat drawings
load complete


C:\Users\pankh\AppData\Local\Temp\ipykernel_14080\1044187459.py:142: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.resize((100, 100), Image.NEAREST)
